In [9]:
import math
import numpy as np
import pandas as pd


In [14]:
dataPanier = pd.read_csv('FichierPanier.csv',encoding ='iso-8859-1',sep=';')
dataPanier['DateAchat']=pd.to_datetime(dataPanier['DateAchat'],format='%d/%m/%Y')
dataPanier.set_index('N° ticket')


,Qté,DateAchat,CodeArticle
N° ticket,,,
10400,1,2019-09-30,TR0024
10400,1,2019-09-30,FR1214
10400,3,2019-09-30,FR1220
10401,2,2019-09-30,TR0031
10401,1,2019-09-30,TR0040
...,...,...,...
11079,2,2019-11-26,TR0023
11080,1,2019-11-26,FR1212
11080,2,2019-11-26,TR0013


In [15]:
dataProduit = pd.read_csv('FichierProduits.csv',encoding ='iso-8859-1',sep=';')
dataProduit.set_index('CodeArticle')

,Famille,DésignationArticle,PrixUnitaire
CodeArticle,,,
CR2001,Crémeux,Vanille,4.0
CR2002,Crémeux,Chocolat,4.0
CR2003,Crémeux,Caramel,4.0
CR2004,Crémeux,Praliné,4.0
CR2005,Crémeux,Tiramisu,4.0
...,...,...,...
TR0511,Tri-goût,Grenadine - Fraise des bois - Menthe,5.0
TR0512,Tri-goût,Grenadine - Anis - Menthe,5.0
TR0513,Tri-goût,Grenadine - Carotte - Menthe,5.0


In [16]:
produitPrix = dataProduit[['Famille','CodeArticle','PrixUnitaire','DésignationArticle']]
panierPrix=dataPanier[['DateAchat','Qté','CodeArticle']]

In [32]:
tmp=pd.merge(dataPanier,dataProduit, on=['CodeArticle'])
tmp['cout'] = tmp.Qté * tmp.PrixUnitaire
tmp['DateAchat']=pd.to_datetime(tmp['DateAchat'])
tmp.set_index('DateAchat')
tmp.dtypes

N° ticket                      int64
Qté                            int64
DateAchat             datetime64[ns]
CodeArticle                   object
Famille                       object
DésignationArticle            object
PrixUnitaire                 float64
cout                         float64
dtype: object

In [24]:
req1 = tmp.groupby("DateAchat").agg({ "cout":sum})
req1.sort_values(by=['DateAchat'])

,cout
DateAchat,
2019-09-01,117.20
2019-09-02,137.65
2019-09-03,124.30
2019-09-04,39.50
2019-09-05,119.30
...,...
2019-11-22,90.35
2019-11-23,512.30
2019-11-24,295.95


In [25]:
req2 = tmp.groupby("Famille").agg({ "cout":sum})
req2

,cout
Famille,
Crémeux,2443.0
Fruités,4846.1
Tri-goût,9830.0


In [28]:
req3 = tmp.groupby("DésignationArticle").agg({ "Qté":sum})
filter1 = req3['Qté']>60
req3[filter1]

,Qté
DésignationArticle,
Amande,72
Cerise,66
Chocolat,67
Citron,63
Fraise,61
Fraise des bois,79
Lait - Citron - Fraise,73
Lait - Mandarine - Citron,64
Lait - Mangue - Cerise,72


In [79]:
tmp4 = tmp[['Qté','Famille','cout']]
tmp4['Day'] = tmp["DateAchat"].dt.day_name()
tmp4.set_index('Day')
req4 = tmp4.groupby(["Day",'Famille']).agg({ "cout":'sum',"Qté":'mean'})
req4


C:\Users\ralex\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


cout       Qté
Day       Famille                    
Friday    Crémeux    385.75  1.725490
          Fruités    622.30  1.567901
          Tri-goût  1220.00  1.605263
Monday    Crémeux    398.75  1.648148
          Fruités    686.00  1.573034
          Tri-goût  1535.00  1.734463
Saturday  Crémeux    394.25  1.745098
          Fruités    823.20  1.541284
          Tri-goût  1730.00  1.609302
Sunday    Crémeux    340.50  1.591837
          Fruités    622.30  1.548780
          Tri-goût  1340.00  1.558140
Thursday  Crémeux    326.25  1.530612
          Fruités    690.90  1.584270
          Tri-goût  1270.00  1.587500
Tuesday   Crémeux    313.50  1.674419
          Fruités    656.60  1.675000
          Tri-goût  1275.00  1.536145
Wednesday Crémeux    284.00  1.585366
          Fruités    744.80  1.600000
          Tri-goût  1460.00  1.586957

In [84]:
req5 = tmp.groupby("DésignationArticle").agg({ "Qté":sum})
filter2 = 6>req5['Qté']
req5[filter2]

,Qté
DésignationArticle,
Cola - Banane - Fraise,1
Lait - Anis - Orange,3
Lait - Anis - Pamplemousse,1
Lait - Banane - Orange,2
Lait - Banane - Pamplemousse,3
Lait - Carotte - Orange,1
Lait - Carotte - Pamplemousse,2
Lait - Coco - Orange,4
Lait - Coco - Pamplemousse,1
